In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import time

start_1 = time.time()

import nltk
import pickle
from string import punctuation
from random import shuffle


nltk.download('floresta')
nltk.download('mac_morpho')


def convert_to_universal_tag(t, reverse=False):
    tagdict = {
#         'n': "NOUN",
#         'num': "NUM",
#         'v-fin': "VERB",
#         'v-inf': "VERB",
#         'v-ger': "VERB",
#         'v-pcp': "VERB",
#         'pron-det': "PRON",
#         'pron-indp': "PRON",
#         'pron-pers': "PRON",
#         'art': "DET",
#         'adv': "ADV",
#         'conj-s': "CONJ",
#         'conj-c': "CONJ",
#         'conj-p': "CONJ",
#         'adj': "ADJ",
#         'ec': "PRT",
#         'pp': "ADP",
#         'prp': "ADP",
#         'prop': "NOUN",
#         'pro-ks-rel': "PRON",
#         'proadj': "PRON",
#         'prep': "ADP",
#         'nprop': "NOUN",
#         'vaux': "VERB",
#         'propess': "PRON",
#         'v': "VERB",
#         'vp': "VERB",
#         'in': "X",
#         'prp-': "ADP",
#         'adv-ks': "ADV",
#         'dad': "NUM",
#         'prosub': "PRON",
#         'tel': "NUM",
#         'ap': "NUM",
#         'est': "NOUN",
#         'cur': "X",
#         'pcp': "VERB",
#         'pro-ks': "PRON",
#         'hor': "NUM",
#         'pden': "ADV",
#         'dat': "NUM",
#         'kc': "ADP",
#         'ks': "ADP",
#         'adv-ks-rel': "ADV",
#         'npro': "NOUN",
    }
    if t in ["N|AP","N|DAD","N|DAT","N|HOR","N|TEL"]:
        t = "NUM"
    if reverse:
        if "|" in t: t = t.split("|")[0]
    else:
        if "+" in t: t = t.split("+")[1]
        if "|" in t: t = t.split("|")[1]
        if "#" in t: t = t.split("#")[0]
    t = t.lower()
    return tagdict.get(t, "." if all(tt in punctuation for tt in t) else t)

# nltk.corpus.mac_morpho.tagged_sents is incorrect, converting tagged_paras to tagged_sents
dataset1 = list(nltk.corpus.floresta.tagged_sents())
dataset2 = [[w[0] for w in sent] for sent in nltk.corpus.mac_morpho.tagged_paras()]


# traindata = [[(w, convert_to_universal_tag(t)) for (w, t) in sent] for sent in dataset1]
# traindata2 = traindata + [[(w, convert_to_universal_tag(t, reverse=True)) for (w, t) in sent] for sent in dataset2]

traindata = [[(w, convert_to_universal_tag(t)) for (w, t) in sent] for sent in dataset1]
traindata2 = traindata + [[(w, convert_to_universal_tag(t, reverse=True)) for (w, t) in sent] for sent in dataset2]


shuffle(traindata)
shuffle(traindata2)

regex_patterns = [
#     (r"^[nN][ao]s?$", "ADP"),
#     (r"^[dD][ao]s?$", "ADP"),
#     (r"^[pP]el[ao]s?$", "ADP"),
#     (r"^[nN]est[ae]s?$", "ADP"),
#     (r"^[nN]um$", "ADP"),
#     (r"^[nN]ess[ae]s?$", "ADP"),
#     (r"^[nN]aquel[ae]s?$", "ADP"),
#     (r"^\xe0$", "ADP"),
]

tagger = nltk.BigramTagger(
            traindata, backoff=nltk.RegexpTagger(
                regex_patterns, backoff=nltk.UnigramTagger(
                    traindata2, backoff=nltk.AffixTagger(
                        traindata2, backoff=nltk.DefaultTagger('n')
                    )
                )
            )
        )
templates = nltk.brill.fntbl37()
tagger = nltk.BrillTaggerTrainer(tagger, templates)
tagger = tagger.train(traindata, max_rules=100)

with open("tagger.pkl", "wb") as f:
    pickle.dump(tagger, f)
    
    
import pandas as pd
import numpy as np
import re
import nltk
import matplotlib.pyplot as plt

pd.options.display.max_colwidth = 200

import re
import os
import pandas as pd

def read_folder(folder_path, texts_list):
  all_files = os.listdir(folder_path)

  for file_name in all_files:
    file = open(folder_path+file_name, 'r+', encoding = "ISO-8859-1")
    text_content = file.read()
    texts_list.append((text_content, file_name))
    file.close()

   
folder_path = '/content/drive/My Drive/Academico/TFG/textos_resumidos/'
texts_list = []
read_folder(folder_path, texts_list)

corpus_df = pd.DataFrame(texts_list, columns = ['text', 'file_name']) 

def normalize_document(doc):
    # lower case and remove special characters\whitespaces
#     doc = re.sub(r'[^a-zA-Z\s]', '', doc, re.I|re.A)
    doc = re.sub("www.\S+","", doc)
    doc = re.sub("\n", " ", doc)
    doc = re.sub('-se', ' se', doc)
    doc = re.sub('-a', ' a', doc)
    doc = re.sub('-o', ' o', doc)
    doc = re.sub('-lhe', ' lhe', doc)
    doc = re.sub('-la', ' la', doc)
    doc = re.sub('-lo', ' lo', doc)
    doc = re.sub('-me', ' me', doc)
    doc = re.sub('-', ' ', doc)
    return doc

normalize_corpus = np.vectorize(normalize_document)

norm_corpus = normalize_corpus(corpus_df['text'])



import pickle
import nltk
nltk.download('punkt')

f = open('tagger.pkl', 'rb')
tagger = pickle.load(f)   
portuguese_sent_tokenizer = nltk.data.load("tokenizers/punkt/portuguese.pickle")


# tamanho de cada frase
# quantidade de frases
# qntd de verbos, pronomes, pontuação por frase e por periodo

from collections import Counter

list_features = []

for text, fn in zip(norm_corpus, corpus_df['file_name'].values):
  tupla = {

      'qnt_frases': 0,
      'media_frase': 0,
      'file_name':'',
      
        'n': 0,
        'num': 0,
        'v-fin': 0,
        'v-inf': 0,
        'v-ger': 0,
        'v-pcp': 0,
        'pron-det': 0,
        'pron-indp': 0,
        'pron-pers': 0,
        'art': 0,
        'adv': 0,
        'conj-s': 0,
        'conj-c': 0,
        'conj-p': 0,
        'adj': 0,
        'ec': 0,
        'pp': 0,
        'prp': 0,
        'prop': 0,
        'pro-ks-rel': 0,
        'proadj': 0,
        'prep': 0,
        'nprop': 0,
        'vaux': 0,
        'propess': 0,
        'v': 0,
        'vp': 0,
        'in': 0,
        'prp-': 0,
        'adv-ks': 0,
        'dad': 0,
        'prosub': 0,
        'tel': 0,
        'ap': 0,
        'est': 0,
        'cur': 0,
        'pcp': 0,
        'pro-ks': 0,
        'hor': 0,
        'pden': 0,
        'dat': 0,
        'kc': 0,
        'ks': 0,
        'adv-ks-rel': 0,
        'npro': 0,
  }
  
  sentences = portuguese_sent_tokenizer.tokenize(text)
  tags = [tagger.tag(nltk.word_tokenize(sentence)) for sentence in sentences]
  print(tags)
  
  tupla['file_name'] = fn
  tupla['qnt_frases'] = len(tags)

  media = 0
  for s in tags:
    media += len(s)
    counter = Counter(elem[1] for elem in s)

    tupla['n'] += counter['n'] or 0
    tupla['num'] += counter['num'] or 0
    tupla['v-fin'] += counter['v-fin'] or 0
    tupla['v-inf'] += counter['v-inf'] or 0
    tupla['v-ger'] += counter['v-ger'] or 0
    tupla['v-pcp'] += counter['v-pcp'] or 0
    tupla['pron-det'] += counter['pron-det'] or 0
    tupla['pron-indp'] += counter['pron-indp'] or 0
    tupla['pron-pers'] += counter['pron-pers'] or 0
    tupla['art'] += counter['art'] or 0
    tupla['adv'] += counter['adv'] or 0
    tupla['conj-s'] += counter['conj-s'] or 0
    tupla['conj-c'] += counter['conj-c'] or 0
    tupla['conj-p'] += counter['conj-p'] or 0
    tupla['adj'] += counter['adj'] or 0
    tupla['ec'] += counter['ec'] or 0
    tupla['pp'] += counter['pp'] or 0
    tupla['prp'] += counter['prp'] or 0
    tupla['prop'] += counter['prop'] or 0
    tupla['pro-ks-rel'] += counter['pro-ks-rel'] or 0
    tupla['proadj'] += counter['proadj'] or 0
    tupla['prep'] += counter['prep'] or 0
    tupla['nprop'] += counter['nprop'] or 0
    tupla['vaux'] += counter['vaux'] or 0
    tupla['propess'] += counter['propess'] or 0
    tupla['v'] += counter['v'] or 0
    tupla['vp'] += counter['vp'] or 0
    tupla['in'] += counter['in'] or 0
    tupla['prp-'] += counter['prp-'] or 0
    tupla['adv-ks'] += counter['adv-ks'] or 0
    tupla['dad'] += counter['dad'] or 0
    tupla['prosub'] += counter['prosub'] or 0
    tupla['tel'] += counter['tel'] or 0
    tupla['ap'] += counter['ap'] or 0
    tupla['est'] += counter['est'] or 0
    tupla['cur'] += counter['cur'] or 0    
    tupla['pcp'] += counter['pcp'] or 0    
    tupla['pro-ks'] += counter['pro-ks'] or 0    
    tupla['hor'] += counter['hor'] or 0    
    tupla['pden'] += counter['pden'] or 0
    tupla['dat'] += counter['dat'] or 0
    tupla['kc'] += counter['kc'] or 0
    tupla['ks'] += counter['ks'] or 0
    tupla['adv-ks-rel'] += counter['adv-ks-rel'] or 0
    tupla['npro'] += counter['npro'] or 0
    tupla['qnt_frases'] += counter['qnt_frases'] or 0
    tupla['media_frase'] += counter['media_frase'] or 0



  tupla['media_frase'] = media
  list_features.append(tupla)
  
entrada = pd.DataFrame(list_features) 

entrada.to_csv('entrada2_tfg.csv')

end_1 = time.time()


print("Time: " + str((end_1 - start_1)))

In [ ]:
import pandas as pd
import numpy as np
import re
import nltk
import matplotlib.pyplot as plt

pd.options.display.max_colwidth = 200

import re
import os
import pandas as pd

def read_folder(folder_path, texts_list):
  all_files = os.listdir(folder_path)

  for file_name in all_files:
    file = open(folder_path+file_name, 'r+', encoding = "ISO-8859-1")
    text_content = file.read()
    texts_list.append((text_content, file_name))
    file.close()

   
folder_path = '/content/drive/My Drive/Academico/TFG/textos_resumidos/'
texts_list = []
read_folder(folder_path, texts_list)

corpus_df = pd.DataFrame(texts_list, columns = ['text', 'file_name']) 

def normalize_document(doc):
    # lower case and remove special characters\whitespaces
#     doc = re.sub(r'[^a-zA-Z\s]', '', doc, re.I|re.A)
    doc = re.sub("www.\S+","", doc)
    doc = re.sub("\n", " ", doc)
    doc = re.sub('-se', ' se', doc)
    doc = re.sub('-a', ' a', doc)
    doc = re.sub('-o', ' o', doc)
    doc = re.sub('-lhe', ' lhe', doc)
    doc = re.sub('-la', ' la', doc)
    doc = re.sub('-lo', ' lo', doc)
    doc = re.sub('-me', ' me', doc)
    doc = re.sub('-', ' ', doc)
    return doc

normalize_corpus = np.vectorize(normalize_document)

norm_corpus = normalize_corpus(corpus_df['text'])

In [ ]:
import pickle
import nltk
nltk.download('punkt')

f = open('tagger.pkl', 'rb')
tagger = pickle.load(f)   
portuguese_sent_tokenizer = nltk.data.load("tokenizers/punkt/portuguese.pickle")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
# tamanho de cada frase
# quantidade de frases
# qntd de verbos, pronomes, pontuação por frase e por periodo

from collections import Counter

list_features = []

for text, fn in zip(norm_corpus, corpus_df['file_name'].values):
  tupla = {

      'qnt_frases': 0,
      'media_frase': 0,
      'file_name':'',
      
        'n': 0,
        'num': 0,
        'v-fin': 0,
        'v-inf': 0,
        'v-ger': 0,
        'v-pcp': 0,
        'pron-det': 0,
        'pron-indp': 0,
        'pron-pers': 0,
        'art': 0,
        'adv': 0,
        'conj-s': 0,
        'conj-c': 0,
        'conj-p': 0,
        'adj': 0,
        'ec': 0,
        'pp': 0,
        'prp': 0,
        'prop': 0,
        'pro-ks-rel': 0,
        'proadj': 0,
        'prep': 0,
        'nprop': 0,
        'vaux': 0,
        'propess': 0,
        'v': 0,
        'vp': 0,
        'in': 0,
        'prp-': 0,
        'adv-ks': 0,
        'dad': 0,
        'prosub': 0,
        'tel': 0,
        'ap': 0,
        'est': 0,
        'cur': 0,
        'pcp': 0,
        'pro-ks': 0,
        'hor': 0,
        'pden': 0,
        'dat': 0,
        'kc': 0,
        'ks': 0,
        'adv-ks-rel': 0,
        'npro': 0,
  }
  
  sentences = portuguese_sent_tokenizer.tokenize(text)
  tags = [tagger.tag(nltk.word_tokenize(sentence)) for sentence in sentences]
  print(tags)
  
  tupla['file_name'] = fn
  tupla['qnt_frases'] = len(tags)

  media = 0
  for s in tags:
    media += len(s)
    counter = Counter(elem[1] for elem in s)

    tupla['n'] += counter['n'] or 0
    tupla['num'] += counter['num'] or 0
    tupla['v-fin'] += counter['v-fin'] or 0
    tupla['v-inf'] += counter['v-inf'] or 0
    tupla['v-ger'] += counter['v-ger'] or 0
    tupla['v-pcp'] += counter['v-pcp'] or 0
    tupla['pron-det'] += counter['pron-det'] or 0
    tupla['pron-indp'] += counter['pron-indp'] or 0
    tupla['pron-pers'] += counter['pron-pers'] or 0
    tupla['art'] += counter['art'] or 0
    tupla['adv'] += counter['adv'] or 0
    tupla['conj-s'] += counter['conj-s'] or 0
    tupla['conj-c'] += counter['conj-c'] or 0
    tupla['conj-p'] += counter['conj-p'] or 0
    tupla['adj'] += counter['adj'] or 0
    tupla['ec'] += counter['ec'] or 0
    tupla['pp'] += counter['pp'] or 0
    tupla['prp'] += counter['prp'] or 0
    tupla['prop'] += counter['prop'] or 0
    tupla['pro-ks-rel'] += counter['pro-ks-rel'] or 0
    tupla['proadj'] += counter['proadj'] or 0
    tupla['prep'] += counter['prep'] or 0
    tupla['nprop'] += counter['nprop'] or 0
    tupla['vaux'] += counter['vaux'] or 0
    tupla['propess'] += counter['propess'] or 0
    tupla['v'] += counter['v'] or 0
    tupla['vp'] += counter['vp'] or 0
    tupla['in'] += counter['in'] or 0
    tupla['prp-'] += counter['prp-'] or 0
    tupla['adv-ks'] += counter['adv-ks'] or 0
    tupla['dad'] += counter['dad'] or 0
    tupla['prosub'] += counter['prosub'] or 0
    tupla['tel'] += counter['tel'] or 0
    tupla['ap'] += counter['ap'] or 0
    tupla['est'] += counter['est'] or 0
    tupla['cur'] += counter['cur'] or 0    
    tupla['pcp'] += counter['pcp'] or 0    
    tupla['pro-ks'] += counter['pro-ks'] or 0    
    tupla['hor'] += counter['hor'] or 0    
    tupla['pden'] += counter['pden'] or 0
    tupla['dat'] += counter['dat'] or 0
    tupla['kc'] += counter['kc'] or 0
    tupla['ks'] += counter['ks'] or 0
    tupla['adv-ks-rel'] += counter['adv-ks-rel'] or 0
    tupla['npro'] += counter['npro'] or 0
    tupla['qnt_frases'] += counter['qnt_frases'] or 0
    tupla['media_frase'] += counter['media_frase'] or 0



  tupla['media_frase'] = media
  list_features.append(tupla)
  
entrada = pd.DataFrame(list_features) 

[[('Acham', 'v'), ('se', 'pron-pers'), ('ai', 'n'), ('uma', 'art'), ('meia', 'n'), ('dúzia', 'n'), ('de', 'prp'), ('rapazes', 'n'), (',', '.'), ('pela', 'n'), ('maior', 'adj'), ('parte', 'n'), ('estudantes', 'n'), (',', '.'), ('desses', 'n'), ('com', 'prp'), ('pretensões', 'n'), ('a', 'prp'), ('estróinas', 'n'), ('e', 'conj-c'), ('excêntricos', 'n'), ('à', 'n'), ('Byron', 'nprop'), (',', '.'), ('e', 'conj-c'), ('que', 'pron-indp'), ('já', 'adv'), ('enfastiados', 'v-pcp'), ('da', 'nprop'), ('sociedade', 'n'), (',', '.'), ('dos', 'n'), ('prazeres', 'n'), ('e', 'conj-c'), ('das', 'n'), ('mulheres', 'n'), (',', '.'), ('costumam', 'v-fin'), ('dizer', 'v-inf'), ('que', 'conj-s'), ('não', 'adv'), ('trocariam', 'v-fin'), ('uma', 'art'), ('fumaça', 'n'), ('de', 'prp'), ('charuto', 'n'), (',', '.'), ('ou', 'conj-c'), ('um', 'art'), ('copo', 'n'), ('de', 'prp'), ('champanha', 'v-fin'), (',', '.'), ('pelo', 'n'), ('mais', 'pron-det'), ('fagueiro', 'n'), ('sorriso', 'n'), ('da', 'nprop'), ('mais', 

In [ ]:
from collections import Counter
import pandas as pd
text = "Um estreito caminho, partindo da porta da casa, cortava o vargedo e ia atravessar o capão e o córrego, por uma pontezinha de madeira, fechada do outro lado por uma tronqueira de varas.Junto à ponte, de um lado e outro do caminho, viam-se duas corpulentas paineiras, cujos galhos, entrelaçando-se no ar, formavam uma arcada de verdura, à entrada do campo onde pastava o gado. Era uma bela tarde de janeiro. Dois meninos brincavam à sombra das paineiras: um rapazinho de doze a treze anos e uma menina, que parecia ser pouco mais nova do que ele. A menina era morena; de olhos grandes, negros e cheios de vivacidade, de corpo esbelto e flexível como o pendão da imbaúba."
tupla = {
    'NOUN': 0,
    'NUM': 0, 
    'VERB': 0, 
    'PRON': 0, 
    'DET': 0, 
    'ADV':0, 
    'CONJ': 0, 
    'PRT': 0, 
    'ADP': 0, 
    'X': 0, 
    '.': 0,
    'qnt_frases': 0,
    'media_frase': 0,
}

sentences = portuguese_sent_tokenizer.tokenize(text)
tags = [tagger.tag(nltk.word_tokenize(sentence)) for sentence in sentences]

tupla['qnt_frases'] = len(tags)

media = 0
for s in tags:
  media += len(s)
  counter = Counter(elem[1] for elem in s)
  tupla['NOUN'] += counter['NOUN'] or 0
  tupla['NUM'] += counter['NUM'] or 0
  tupla['VERB'] += counter['VERB'] or 0
  tupla['PRON'] += counter['PRON'] or 0
  tupla['DET'] += counter['DET'] or 0
  tupla['ADV'] += counter['ADV'] or 0
  tupla['CONJ'] += counter['CONJ'] or 0
  tupla['ADP'] += counter['ADP'] or 0
  tupla['X'] += counter['X'] or 0
  tupla['.'] += counter['.'] or 0
  tupla['qnt_frases'] += counter['qnt_frases'] or 0
  tupla['media_frase'] += counter['media_frase'] or 0


media = media/len(tags)
tupla['media_frase'] = media

  
teste = pd.DataFrame([tupla]) 

In [ ]:
entrada.to_csv('entrada2_tfg.csv')